In [1]:
# import the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline
import cv2
import imutils
from imutils import paths
import pytesseract
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
from skimage.segmentation import clear_border
import warnings
warnings.simplefilter('ignore')

In [2]:
class computervisionANPR:
    # store the minimum and maximum rectangular aspect ratio
    def __init__(self,minAR=4,maxAR=5,debug=False):
        self.minAR=minAR
        self.maxAR=maxAR
        self.debug=debug # A flag to indicate whether we should display intermediate results in our image processing pipeline
        
    def debug_imshow(self,title,image,waitKey=False):
        if self.debug:
            cv2.imshow(title,image)
            
            if waitKey:
                cv2.waitKey(0)
                
    def locate_licence_plate_candidates(self,gray,keep=5):
        # perform a blackhat morphological operation
        rectKernel=cv2.getStructuringElement(cv2.MORPH_RECT,(13,5))
        blackhat=cv2.morphologyEx(gray,cv2.MORPH_BLACKHAT,rectKernel)
        self.debug_imshow('Blackhat',blackhat)
        
        # next, find regions in the image that are light
        sqrKernel=cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
        light=cv2.morphologyEx(gray,cv2.MORPH_CLOSE,sqrKernel)
        light=cv2.threshold(light,0,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)[1]
        self.debug_imshow('Light Regions',light)
        
        grad=cv2.Sobel(blackhat,ddepth=cv2.CV_32F,dx=1,dy=0,ksize=-1)
        grad=np.absolute(grad)
        (minVal,maxVal)=(np.min(grad),np.max(grad))
        grad=(grad-minVal)/(maxVal-minVal)
        grad=(grad*255).astype('uint8')
        self.debug_imshow('Gradient',grad)
        
        # blur the gradient representation, applying a closing
        # operation, and threshold the image using Otsu's method
        
        grad=cv2.GaussianBlur(grad,(5,5),0)
        grad=cv2.morphologyEx(grad,cv2.MORPH_CLOSE,sqrKernel)
        thresh=cv2.threshold(grad,0,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)[1]
        self.debug_imshow('Gradient Threshold',thresh)
        
        thresh=cv2.erode(thresh,None,iterations=2)
        thresh=cv2.dilate(thresh,None,iterations=2)
        self.debug_imshow('Gradient erode/dilate',thresh)
        
        thresh=cv2.bitwise_and(thresh,thresh,mask=light)
        thresh=cv2.erode(thresh,None,iterations=2)
        thresh=cv2.dilate(thresh,None,iterations=1)
        self.debug_imshow('Final',thresh,waitKey=True)
        
        # find contours in the thresholded image and sort them by
        # their size in descending order, keeping only the largest
        # ones
        
        cnts=cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnts=imutils.grab_contours(cnts)
        cnts=sorted(cnts,key=cv2.contourArea,reverse=True)[:keep]
        
        return cnts
    #Pruning license plate candidates
    def locate_licence_plate(self,gray,candidates,clearBorder=False):
        lpCnt=None
        roi=None
        
        for c in candidates:
            (x,y,w,h)=cv2.boundingRect(c)
            ar=w/float(h)
            
            if ar>=self.minAR and ar<=self.maxAR:
                lpCnt=c
                
                licenceplate=gray[y:y+h,x:x+w]
                roi=cv2.threshold(licenceplate,0,255,cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)[1]
                
                if clearBorder:
                    roi=clearBorder(roi)
                    
                self.debug_imshow('Licence Plate', licenceplate)
                self.debug_imshow('ROI',roi,waitKey=True)
                
        return (roi,lpCnt)
    
    #Defining Tesseract ANPR options including an OCR Character Whitelist and Page Segmentation Mode (PSM)
    def build_tesseract_options(self,psm=7):
        alphanumeric='ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
        options="-c tessedit_char_whitelist={}".format(alphanumeric)
        
        options+="--psm {}".format(psm)
        
        return options
    #The Central Method
    def find_and_ocr(self,image,psm=7,clearBorder=False):
        
        lpText=None
        
        gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        candidates=self.locate_licence_plate_candidates(gray)
        (lp,lpCnt)=self.locate_licence_plate(gray,candidates,clearBorder=clearBorder)
        
        if lp is not None:
            options=self.build_tesseract_options(psm=psm)
            lpText=pytesseract.image_to_string(lp,config=options)
            self.debug_imshow('Licence Plate',lp)
            
        return (lpText,lpCnt)

In [1]:
# strip out non-ASCII text so we can draw the text on the image
# using OpenCV
def cleanup_text(text):
    return ''.join([c if ord(c)<128 else '' for c in text]).strip()

In [4]:
args={'input':r"F:\Computer Vision 2\ANPR OCR\archive (4)\Indian_Number_Plates\Sample_Images",
    'clear-border':-1,
     'psm':7,
      'debug':-1}

In [5]:
anpr=computervisionANPR(debug=args['debug']>0)

imagePaths=sorted(list(paths.list_images(args['input'])))

# loop over all image paths in the input directory
for imagePath in imagePaths:
    image=cv2.imread(imagePath)
    image=imutils.resize(image,width=600)
    
    (lpText,lpCnt)=anpr.find_and_ocr(image,psm=args['psm'],clearBorder=args['clear-border']>0)
    
    if lpText is not None and lpCnt is not None:
        box=cv2.boxPoints(cv2.minAreaRect(lpCnt))
        box=box.astype('int')
        cv2.drawContours(image,[box],-1,(0,255,0),2)
        
        (x,y,w,h)=cv2.boundingRect(lpCnt)
        cv2.putText(image,cleanup_text(lpText),(x,y-15),cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,255,0),2)
        
        # show the output ANPR image
        print("[INFO] {}".format(lpText))
        cv2.imshow('output ANPR', image)
        cv2.waitKey(0)

-1